In [22]:
import pandas as pd
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, DataCollatorWithPadding
import random
from torch.utils.data import DataLoader

In [2]:
raw_code_url = '/data/dataset/CodeXGLUE/Code-Code/Clone-detection-BigCloneBench/dataset/data_enhanced.jsonl'
raw_code = pd.read_json(path_or_buf=raw_code_url, lines=True)
raw_code = raw_code.set_index('idx')

In [19]:
raw_code['ast_des'] = raw_code['ast'] + raw_code['description']
raw_code

,func,description,ast,ast_des
idx,,,,
10000832,public static void main(String[] args) {\n...,Throws a SecurityException if the calling thre...,MethodDeclaration Modifier static public main ...,MethodDeclaration Modifier static public main ...
10005623,public synchronized String getSerialNumber...,Gets the next Request instance for which a res...,MethodDeclaration Modifier synchronized public...,MethodDeclaration Modifier synchronized public...
10005624,public Object run() {\n ...,An abstract implementation of a calendar syste...,MethodDeclaration Modifier public ReferenceTyp...,MethodDeclaration Modifier public ReferenceTyp...
10005674,public String post() {\n if (conten...,Gets a representation of the current choice as...,MethodDeclaration Modifier public ReferenceTyp...,MethodDeclaration Modifier public ReferenceTyp...
10005879,@Override\n public void onCreate(Bundle...,"Hides the splash screen, closes the window, an...",MethodDeclaration Modifier public Annotation O...,MethodDeclaration Modifier public Annotation O...
...,...,...,...,...
9980885,private void verifyAvailability() {\n ...,Returns the current time in milliseconds.\nCal...,MethodDeclaration Modifier private verifyAvail...,MethodDeclaration Modifier private verifyAvail...
9983757,public static void copyFiles(String strPat...,Creates the directory named by this abstract p...,MethodDeclaration Modifier static public copyF...,MethodDeclaration Modifier static public copyF...
9983984,private String SHA1(String text) throws No...,Deprecated.This method does not properly conve...,MethodDeclaration Modifier private ReferenceTy...,MethodDeclaration Modifier private ReferenceTy...


In [54]:
sum1 = 0
num = 0
for code in raw_code['ast_des']:
    sum1 += len(code.split(' '))
    num += 1
print(sum1/num)

341.20304624150776


In [17]:
checkpoint = 'microsoft/codebert-base'
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
model = RobertaModel.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [7]:
raw_code_index = raw_code.index.tolist()

over_length_ids = [10000832, 10151623, 10540676, 10690321, 10717656, 10793825, 10934628, 11339042, 11603577, 11940679, 12082150, 12119068, 12335897, 12352751, 12389873, 12415477, 12838273, 12838274, 12914531, 13099033, 13292215, 13292580, 13456795, 1349815, 13563706, 13586003, 13644040, 13650923, 13747998, 13776078, 13961100, 14020143, 14190765, 14310068, 14661394, 1477292, 14794142, 15295408, 15453012, 15723802, 15758923, 16002345, 16002347, 16006791, 16142591, 1637147, 16518661, 17314208, 17551920, 17573144, 17750515, 17829989, 17874921, 17917053, 18284812, 18433984, 18574455, 18790182, 18822890, 19074021, 19090289, 19276021, 19276022, 19340788, 19382420, 19434890, 19434892, 1944490, 19478367, 19556732, 1962490, 19634773, 19841853, 19942676, 20122631, 20275058, 2067794, 20833509, 20856391, 20885480, 21028028, 21161120, 21318345, 21493541, 2157431, 21652119, 22031237, 22033685, 22035132, 22114133, 22222255, 2235431, 2247987, 22580642, 22673614, 2285441, 23041161, 23094550, 23188198, 23248619, 23370708, 23510383, 23611768, 2450, 2476569, 2511576, 2769195, 2771573, 2771574, 285947, 2996859, 325062, 3375714, 3375715, 3375723, 3400236, 37044, 3867253, 416857, 4420769, 4494367, 4581365, 4660318, 4681906, 4780347, 4792385, 4854974, 5021563, 5189131, 5252227, 5389524, 5430189, 552318, 557726, 5620476, 564191, 5691586, 592597, 6008880, 6121196, 6147227, 6304372, 6304373, 6333737, 6403868, 6644748, 6961579, 6966398, 7005223, 7149578, 726690, 7300257, 7300264, 7300267, 733283, 7394826, 7665877, 7687037, 7727956, 793122, 8079516, 8109022, 8335944, 83802, 8581437, 8641070, 9050003, 9221721, 9261908, 9530015, 9581835, 961457, 9647574, 9687064, 9705209, 9782243, 979163, 98309, 98428, 9980609]
# over_length_ids = [10690321, 10717656, 10793825, 11339042, 12119068, 12352751, 12415477, 13099033, 15723802, 1637147, 16518661, 17750515, 18822890, 19276021, 2067794, 20833509, 20856391, 20885480, 21028028, 22033685, 22035132, 2235431, 2247987, 22580642, 2285441, 23094550, 23248619, 2996859, 325062, 3375723, 3867253, 4494367, 4660318, 4780347, 5021563, 5189131, 5252227, 564191, 6147227, 7005223, 7300267, 793122, 8109022, 8581437, 9221721, 9647574, 98309]

def read_ccd_pairs(url):
    data = []
    with open(url) as f:
        for line in f:
            line = line.strip()
            id1, id2, label = line.split('\t')
            if int(id1) not in raw_code_index or int(id2) not in raw_code_index or int(id1) in over_length_ids or int(id2) in over_length_ids:
                continue
            label = 0 if label == '0' else 1
            data.append((int(id1), int(id2), label))
    return data

In [8]:
train_url = '/data/dataset/CodeXGLUE/Code-Code/Clone-detection-BigCloneBench/dataset/train.txt'
valid_url = '/data/dataset/CodeXGLUE/Code-Code/Clone-detection-BigCloneBench/dataset/test.txt'
test_url = '/data/dataset/CodeXGLUE/Code-Code/Clone-detection-BigCloneBench/dataset/valid.txt'

In [9]:
train_data = read_ccd_pairs(train_url)
valid_data = read_ccd_pairs(valid_url)
test_data = read_ccd_pairs(test_url)

In [12]:
random.seed(666)
train_data_small = random.sample(train_data, 10000)
valid_data_small = random.sample(valid_data, 5000)
test_data_small = random.sample(test_data, 5000)

In [36]:
def get_raw_code(data):
    code1 = []
    code2 = []
    label = []
    for pair in data:
        code1.append(raw_code['ast_des'][pair[0]])
        # print(len(raw_code['ast_des'][pair[0]]))
        code2.append(raw_code['ast_des'][pair[1]])
        # print(len(raw_code['ast_des'][pair[1]]))
        label.append(pair[2])
    return {'code1': code1, 'code2': code2, 'label': label}

In [ ]:
train_code = get_raw_code(train_data_small)
valid_code = get_raw_code(valid_data_small)
test_code = get_raw_code(test_data_small)

In [52]:
length = []
for ast_des in train_data['func']:
    length.append(len(tokenizer(ast_des).input_ids))

sum(length)/len(length)

786.974030243261

702.0207100591716